In [1]:
import requests
from pprint import pprint
import sqlite3

In [2]:
# API key you received from NREL
api_key = "RjZpx6DxeHBP4d17uK9Uifu6qhas3674psy6dJ7Q"



In [3]:
# Base URL for the NREL API
base_url = "https://developer.nrel.gov"
endpoint = "/api/alt-fuel-stations/v1.json"

url = base_url + endpoint



In [4]:
# Set up parameters: Electric Stations in Texas
params = {
    'api_key': api_key,
    'limit': 1000,  # Limiting the number of results returned
    'fuel_type': 'ELEC',
    'state': 'TX'
}



In [5]:
conn = sqlite3.connect('fuel_stations.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS fuel_stations (
        id INTEGER PRIMARY KEY,
        station_name TEXT,
        address TEXT,
        fuel_type_code TEXT,
        ev_network TEXT,
        ev_level1_evse_num INTEGER,
        ev_level2_evse_num INTEGER,
        ev_dc_fast_num INTEGER,
        ev_connector_types TEXT,
        ev_pricing TEXT,
        ev_renewable_source INTEGER,
        restricted_access INTEGER,
        state TEXT,
        city TEXT,
        zip TEXT,
        latitude REAL,
        longitude REAL
    )
''')
conn.commit()

In [6]:
# Make a GET request to the API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    json_data = response.json()

    # Insert each record into the table
    for station in json_data['fuel_stations']:
        ev_connector_types = ','.join(station['ev_connector_types'])
        cursor.execute('''
            INSERT INTO fuel_stations (id, station_name, address, fuel_type_code, ev_network, ev_level1_evse_num, ev_level2_evse_num, ev_dc_fast_num, ev_connector_types, ev_pricing, ev_renewable_source, restricted_access, state, city, zip, latitude, longitude)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            station['id'],
            station['station_name'],
            station['street_address'],
            station['fuel_type_code'],
            station['ev_network'],
            station['ev_level1_evse_num'],
            station['ev_level2_evse_num'],
            station['ev_dc_fast_num'],
            ev_connector_types,
            station['ev_pricing'],
            station['ev_renewable_source'],
            station['restricted_access'],
            station['state'],
            station['city'],
            station['zip'],
            station['latitude'],
            station['longitude']
        ))

    # Commit the changes and close the connection
    conn.commit()
else:
    print(f"Failed to retrieve data: {response.status_code}")

Failed to retrieve data: 422
